In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import geoviews.tile_sources as gts
from cartopy import crs
import dask

In [ ]:
gv.extension('bokeh')
%output size=300

In [ ]:
dask.config.set(scheduler='threaded')

In [ ]:
rsync -v --progress  --exclude provenance -a /mnt/oneclient/eWaterCycle/forecast-results/forecasts ~/

In [ ]:
files = glob.glob('/mnt/home/student1/forecasts/20181106/member*-discharge_dailyTot_output.nc')

In [ ]:
ensemble_members = [xr.open_dataset(fn) for fn in files]

In [ ]:
ds = xr.concat(ensemble_members, pd.Index(range(1,21), name='ensemble_id'))

In [14]:
ds

<xarray.Dataset>
Dimensions:      (ensemble_id: 20, lat: 360, lon: 720, time: 9)
Coordinates:
  * time         (time) datetime64[ns] 2018-11-06 2018-11-07 2018-11-08 ...
  * lat          (lat) float32 89.75 89.25 88.75 88.25 87.75 87.25 86.75 ...
  * lon          (lon) float32 -179.75 -179.25 -178.75 -178.25 -177.75 ...
  * ensemble_id  (ensemble_id) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
Data variables:
    discharge    (ensemble_id, time, lat, lon) float32 nan nan nan nan nan ...
Attributes:
    description:  test version (by Niels Drost)
    institution:  Department of Physical Geography, Utrecht University
    title:        PCR-GLOBWB output

In [ ]:
dataset = gv.Dataset(ds.where(ds.discharge > 100))

In [ ]:
# Discharge over time plot bewlo for the green dot in the map
point = (-60.25, -10.25)

In [ ]:
gts.CartoMidnight * dataset.select(ensemble_id=1).to(gv.Image, ['lon', 'lat']).options(tools=['hover']) * gv.Points(point, crs=crs.PlateCarree()).options(color='green')

In [ ]:
point_ds = gv.Dataset(ds).select(lon=point[0], lat=point[1])
point_ds.to(hv.Curve).reindex().overlay() * hv.ErrorBars(point_ds.aggregate('time', np.mean, np.std))
